In [1]:
!pip install lightfm
import lightfm
import pandas as pd
import numpy as np
import time

C:\Users\suyeon\miniconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
books = pd.read_csv('../../data/books.csv')
users = pd.read_csv('../../data/users.csv')
ratings = pd.read_csv('../../data/train_ratings.csv')
users.head()

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0


In [3]:
books.shape

(149570, 10)

In [4]:
# user_rating = ratings.merge(users, how='left',left_on = 'user_id', right_on = 'user_id')
# df = user_rating.merge(books, how='left', left_on = 'isbn', right_on = 'isbn')
# df.head()

In [5]:
# df.isnull().sum()

In [6]:
users.isna().sum()

user_id         0
location        0
age         27833
dtype: int64

In [7]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

C:\Users\suyeon\AppData\Local\Temp\ipykernel_4208\3844664685.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [8]:
users.head()

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins,ontario,canada",NaN,timmins,ontario,canada
1,11400,"ottawa,ontario,canada",49.0,ottawa,ontario,canada
2,11676,"na,na,na",NaN,NaN,NaN,NaN
3,67544,"toronto,ontario,canada",30.0,toronto,ontario,canada
4,85526,"victoria,britishcolumbia,canada",36.0,victoria,britishcolumbia,canada


In [9]:
users['age'] = users['age'].fillna(users.mean()['age'])

C:\Users\suyeon\AppData\Local\Temp\ipykernel_4208\3978783657.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  users['age'] = users['age'].fillna(users.mean()['age'])


In [10]:
users.isna().sum()

user_id                0
location               0
age                    0
location_city        122
location_state      3254
location_country    2124
dtype: int64

In [11]:
users.head()

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins,ontario,canada",36.069873,timmins,ontario,canada
1,11400,"ottawa,ontario,canada",49.000000,ottawa,ontario,canada
2,11676,"na,na,na",36.069873,NaN,NaN,NaN
3,67544,"toronto,ontario,canada",30.000000,toronto,ontario,canada
4,85526,"victoria,britishcolumbia,canada",36.000000,victoria,britishcolumbia,canada


In [12]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

In [13]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

In [14]:
users.isna().sum()

user_id                0
location               0
age                    0
location_city        122
location_state      1132
location_country     271
dtype: int64

In [15]:
users = users.drop(['location'], axis=1)
users.head()

,user_id,age,location_city,location_state,location_country
0,8,36.069873,timmins,ontario,canada
1,11400,49.000000,ottawa,ontario,canada
2,11676,36.069873,NaN,NaN,NaN
3,67544,30.000000,toronto,ontario,canada
4,85526,36.000000,victoria,britishcolumbia,canada


In [16]:
users_copy = users.copy()

In [17]:
col = ['user_id','age','location_country','location_state','location_city']
users = pd.DataFrame(columns=col)
users['user_id'] = users_copy['user_id']
users['age'] = users_copy['age']
users['location_country'] = users_copy['location_country']
users['location_state'] = users_copy['location_state']
users['location_city'] = users_copy['location_city']
users.head()

,user_id,age,location_country,location_state,location_city
0,8,36.069873,canada,ontario,timmins
1,11400,49.000000,canada,ontario,ottawa
2,11676,36.069873,NaN,NaN,NaN
3,67544,30.000000,canada,ontario,toronto
4,85526,36.000000,canada,britishcolumbia,victoria


In [18]:
users = users.dropna()

In [19]:
users.isna().sum()

user_id             0
age                 0
location_country    0
location_state      0
location_city       0
dtype: int64

In [20]:
users.head()

,user_id,age,location_country,location_state,location_city
0,8,36.069873,canada,ontario,timmins
1,11400,49.000000,canada,ontario,ottawa
3,67544,30.000000,canada,ontario,toronto
4,85526,36.000000,canada,britishcolumbia,victoria
5,96054,29.000000,canada,ontario,ottawa


In [21]:
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)

In [22]:
publisher_count_df.head()

,publisher,count
0,Harlequin,3005
1,Ballantine Books,2322
2,Pocket,2274
3,Penguin Books,1943
4,Bantam Books,1938


In [23]:
books[books['publisher']=="Penguin Books"]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
68,0140067477,The Tao of Pooh,Benjamin Hoff,1983.0,Penguin Books,http://images.amazon.com/images/P/0140067477.0...,en,['Humor'],An expert in Chinese philosophy explains facet...,images/0140067477.01.THUMBZZZ.jpg
119,0143034359,Thursday Next in the Well Of Lost Plots (Thurs...,Fforde Jasper,2004.0,Penguin Books,http://images.amazon.com/images/P/0143034359.0...,en,['Fiction'],Exhausted by her stint as Miss Havisham&#39;s ...,images/0143034359.01.THUMBZZZ.jpg
157,0140250964,Snow Angels,Stewart O'Nan,1995.0,Penguin Books,http://images.amazon.com/images/P/0140250964.0...,en,['Fiction'],Weaves together two stories in the account of ...,images/0140250964.01.THUMBZZZ.jpg
196,0140272100,Vanished,Mary McGarry Morris,1997.0,Penguin Books,http://images.amazon.com/images/P/0140272100.0...,en,['Fiction'],Aubrey Wallace is the kind of man no one notices.,images/0140272100.01.THUMBZZZ.jpg
298,014028009X,Bridget Jones's Diary,Helen Fielding,1999.0,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,en,['Fiction'],This witty national bestseller is now availabl...,images/014028009X.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149154,0140423818,Lord Byron: Selected Poems (Penguin Classics),George Gordon Byron,1996.0,Penguin Books,http://images.amazon.com/images/P/0140423818.0...,en,['Poetry'],&quot; &quot;Byron&#39;s contemporary populari...,images/0140423818.01.THUMBZZZ.jpg
149301,0140290788,Time Out Chicago (Time Out Chicago),Time Out,2000.0,Penguin Books,http://images.amazon.com/images/P/0140290788.0...,NaN,NaN,NaN,images/0140290788.01.THUMBZZZ.jpg
149345,0140182551,The Old Wives' Tale (20th Century Classics),Arnold Bennett,1990.0,Penguin Books,http://images.amazon.com/images/P/0140182551.0...,en,['Fiction'],Victorian novel set in the pottery district of...,images/0140182551.01.THUMBZZZ.jpg
149387,0140390715,The Pathfinder (Penguin Classic),James Fenimore Cooper,1989.0,Penguin Books,http://images.amazon.com/images/P/0140390715.0...,en,['Fiction'],Cooper undertakes a &quot;hazardous experiment...,images/0140390715.01.THUMBZZZ.jpg


In [24]:
books['publisher'].nunique() # 수정전 항목 수를 확인합니다.

11571

In [25]:
books[books['isbn'].apply(lambda x: x[:4])=='0140']['publisher'].unique()

array(['Penguin Books', 'Penguin Books Ltd', 'Puffin Books',
       'Penguin USA (Paper)', 'Penguin USA', 'Puffin',
       'Penguin Putnam~trade', 'Penguin Putnam~mass',
       'Penguin/Puffin Mass Market (Mm)', 'Viking Books',
       'Penguin Books Australia Ltd', 'Penguin Uk',
       'Scott Foresman (Pearson K-12)',
       'Penguin Putnam Books for Young Readers',
       'French & European Pubns', 'Viking Penguin', 'Arkana',
       'Penguin Highbridge Audio', 'Penguin (Non-Classics)',
       'Penguin Books, Limited (UK)', 'Viking Press', 'Penguin U S A',
       'Penguin/Puffin Mass Market', 'Select Penguin (Mm)',
       'Penguin Putnam', 'Penguin Books India', 'Select Penguin',
       'Penguin Books Australia Ltd.', 'Viking Penguin Inc',
       'Penguin Audiobooks', 'Studio Books', 'Penguin',
       'Hyperion Books for Children', 'Putnam Pub Group', 'Penguin India',
       'Compass Books', 'Penguin Putnam~childrens Hc',
       'Penguin in association with Secker and Warburg', 'Time O

In [26]:
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values

In [27]:
for publisher in modify_list:
    try:
        number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
        right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
        books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
    except: 
        pass

In [28]:
books['publisher'].nunique() #수정 후 출판사 갯수입니다

1523

In [29]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
dtype: int64

In [30]:
import re
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())

In [31]:
books['category'].value_counts()

Fiction                                            32956
Juvenile Fiction                                    5804
Biography Autobiography                             3320
History                                             1925
Religion                                            1818
                                                   ...  
Eschatology                                            1
Dogsledding                                            1
Brenner Simon Fictitious character                     1
Gone with the wind Motion picture 1939                 1
Authors Canadian English 20th century Biography        1
Name: category, Length: 4137, dtype: int64

In [32]:
books['category'] = books['category'].str.lower()

In [33]:
books['category'].value_counts()

fiction                                            33016
juvenile fiction                                    5835
biography autobiography                             3326
history                                             1927
religion                                            1818
                                                   ...  
brenner simon fictitious character                     1
gone with the wind motion picture 1939                 1
disguise                                               1
country lawyers                                        1
authors canadian english 20th century biography        1
Name: category, Length: 4105, dtype: int64

In [34]:
category_df = pd.DataFrame(books['category'].value_counts()).reset_index()
category_df.columns = ['category','count']
category_df.head()

,category,count
0,fiction,33016
1,juvenile fiction,5835
2,biography autobiography,3326
3,history,1927
4,religion,1818


In [35]:
category_df[category_df['count']>=10]

,category,count
0,fiction,33016
1,juvenile fiction,5835
2,biography autobiography,3326
3,history,1927
4,religion,1818
...,...,...
253,children s poetry english,10
254,abused children,10
255,courage,10
256,diary fiction,10


In [36]:
books['category_high'] = books['category'].copy()
books.loc[books[books['category']=='biography'].index, 'category_high'] = 'biography autobiography'
books.loc[books[books['category']=='autobiography'].index,'category_high'] = 'biography autobiography'

In [37]:
books[books['category'].str.contains('history', na=False)]['category'].unique()

array(['history', 'oral history', 'natural history', 'history modern',
       'world history', 'politics history 1945', 'history asia',
       'church history', 'constitutional history'], dtype=object)

In [38]:
books.loc[books[books['category'].str.contains('history',na=False)].index,'category_high'] = 'history'

In [39]:
categories = ['garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category_high'] = category

In [40]:
category_high_df = pd.DataFrame(books['category_high'].value_counts()).reset_index()
category_high_df.columns = ['category','count']
category_high_df.head(10)

,category,count
0,fiction,39678
1,biography autobiography,3335
2,history,1949
3,religion,1824
4,nonfiction,1427
5,humor,1291
6,social,1271
7,business,1146
8,cook,1125
9,body mind spirit,1113


In [41]:
# 5개 이하인 항목은 others로 묶어주도록 하겠습니다.
others_list = category_high_df[category_high_df['count']<5]['category'].values

In [42]:
books.loc[books[books['category_high'].isin(others_list)].index, 'category_high']='others'

In [43]:
books['category'].nunique()

4105

In [44]:
books['category_high'].nunique()

412

In [45]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
category_high          68851
dtype: int64

In [46]:
books = books.dropna()

In [47]:
books.isna().sum()

isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
img_url                0
language               0
category               0
summary                0
img_path               0
category_high          0
dtype: int64

In [48]:
user_rating = ratings.merge(users, how='left',left_on = 'user_id', right_on = 'user_id')
df = user_rating.merge(books, how='left', left_on = 'isbn', right_on = 'isbn')
df.head()

,user_id,isbn,rating,age,location_country,location_state,location_city,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high
0,8,0002005018,4,36.069873,canada,ontario,timmins,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses
1,67544,0002005018,7,30.000000,canada,ontario,toronto,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses
2,123629,0002005018,8,36.069873,canada,ontario,kingston,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses
3,200273,0002005018,8,36.069873,canada,ontario,comber,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses
4,210926,0002005018,9,36.069873,canada,ontario,guelph,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses


In [49]:
userid_unique = df['user_id'].unique()
book_unique = df['isbn'].unique()
print(len(userid_unique))

59803


In [50]:
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
book_to_idx = {v:k for k,v in enumerate(book_unique)}

In [51]:
df.isna().sum()

user_id                     0
isbn                        0
rating                      0
age                     10300
location_country        10300
location_state          10300
location_city           10300
book_title             121221
book_author            121221
year_of_publication    121221
publisher              121221
img_url                121221
language               121221
category               121221
summary                121221
img_path               121221
category_high          121221
dtype: int64

In [52]:
df = df.dropna()

In [53]:
df.isna().sum()

user_id                0
isbn                   0
rating                 0
age                    0
location_country       0
location_state         0
location_city          0
book_title             0
book_author            0
year_of_publication    0
publisher              0
img_url                0
language               0
category               0
summary                0
img_path               0
category_high          0
dtype: int64

In [54]:
df['uid'] = df['user_id'].map(user_to_idx)
df['iid'] = df['isbn'].map(book_to_idx)
df.head()

,user_id,isbn,rating,age,location_country,location_state,location_city,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high,uid,iid
0,8,0002005018,4,36.069873,canada,ontario,timmins,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,0,0
1,67544,0002005018,7,30.000000,canada,ontario,toronto,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,1,0
2,123629,0002005018,8,36.069873,canada,ontario,kingston,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,2,0
3,200273,0002005018,8,36.069873,canada,ontario,comber,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,3,0
4,210926,0002005018,9,36.069873,canada,ontario,guelph,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,4,0


In [55]:
# df.insert(1, 'location_city', users['location_city'])
# df.insert(2, 'location_state', users['location_state'])
# df.insert(3, 'location_country', users['location_country'])
# df.head()

In [56]:
data = df.copy().reset_index(drop=True)
data.columns = data.columns.str.lower() # 나중에 뺄 것
data.head()

,user_id,isbn,rating,age,location_country,location_state,location_city,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high,uid,iid
0,8,0002005018,4,36.069873,canada,ontario,timmins,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,0,0
1,67544,0002005018,7,30.000000,canada,ontario,toronto,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,1,0
2,123629,0002005018,8,36.069873,canada,ontario,kingston,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,2,0
3,200273,0002005018,8,36.069873,canada,ontario,comber,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,3,0
4,210926,0002005018,9,36.069873,canada,ontario,guelph,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,4,0


In [57]:
data.isnull().sum()

user_id                0
isbn                   0
rating                 0
age                    0
location_country       0
location_state         0
location_city          0
book_title             0
book_author            0
year_of_publication    0
publisher              0
img_url                0
language               0
category               0
summary                0
img_path               0
category_high          0
uid                    0
iid                    0
dtype: int64

In [58]:
data.head()

,user_id,isbn,rating,age,location_country,location_state,location_city,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high,uid,iid
0,8,0002005018,4,36.069873,canada,ontario,timmins,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,0,0
1,67544,0002005018,7,30.000000,canada,ontario,toronto,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,1,0
2,123629,0002005018,8,36.069873,canada,ontario,kingston,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,2,0
3,200273,0002005018,8,36.069873,canada,ontario,comber,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,3,0
4,210926,0002005018,9,36.069873,canada,ontario,guelph,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,actresses,4,0


In [59]:
data.shape

(179633, 19)

In [60]:
# ratings_source : build_interactions 에 사용
# --> [(user1, item1), (user2, item5), ... ] 
ratings_source = [(data['uid'][i], data['iid'][i]) for i in range(data.shape[0])]

In [61]:
# item_source: build_item_features 에 사용
# --> [(item1, [feature, feature, ...]), (item2, [feature, feature, ...])]
item_meta = data[['iid', 'category','language','book_author','publisher']]
item_source = [(item_meta['iid'][i],
                        [item_meta['category'][i],
                         item_meta['language'][i],
                         item_meta['book_author'][i],
                         item_meta['publisher'][i]]) for i in range(item_meta.shape[0])]

In [62]:
# user_source: build_iutem_features 에 사용
# --> [(user1, [feature, feature, ...]), (user2, [feature, feature, ...])]
user_meta = data[['uid','age','location_city','location_state','location_country']]

user_source = [(user_meta['uid'][i],
                [user_meta['age'][i],
                 user_meta['location_city'][i],
                 user_meta['location_state'][i],
                 user_meta['location_country'][i]]) for i in range(user_meta.shape[0])]

In [63]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(users=data['uid'].unique(),
            items=data['iid'].unique(),
            item_features=item_meta[item_meta.columns[1:]].values.flatten(),
            user_features=user_meta[user_meta.columns[1:]].values.flatten()
            )

In [64]:
item_meta.columns[1:]

Index(['category', 'language', 'book_author', 'publisher'], dtype='object')

In [65]:
interactions, weights = dataset.build_interactions(ratings_source)
item_features = dataset.build_item_features(item_source)
user_features = dataset.build_user_features(user_source)

In [66]:
from lightfm.cross_validation import random_train_test_split 

In [67]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train.toarray()
test.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [68]:
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()

In [69]:
from lightfm import LightFM

In [70]:
model = LightFM(no_components=10,
              learning_schedule='adagrad',
              loss='warp',
              learning_rate=0.05,
              random_state=42)

In [73]:
model.fit(interactions=train,
          item_features=item_features,
          user_features = user_features, 
          sample_weight=train_weights,
          epochs=100,
          verbose=False)

In [74]:
scores = []

for item_id in range(27204):
    user_ids = np.array([1])
    item_ids = np.array([item_id])
    s = model.predict(user_ids, item_ids, 
                      item_features=item_features, user_features=user_features)
    scores.append([item_ids[0], s[0]])

In [110]:
user1_score = pd.DataFrame(scores, columns=['item_id', 'score'])
user1_score.head()

,item_id,score
0,0,-49.394451
1,1,-54.527943
2,2,-53.670879
3,3,-51.295696
4,4,-54.265755


In [76]:
!pip install sklearn
from sklearn.preprocessing import MinMaxScaler

In [77]:
normalizationFeatureScaling = MinMaxScaler()
normalizationScaling = normalizationFeatureScaling.fit_transform(user1_score.drop(['item_id'], axis=1))

In [79]:
normalizationScaling = pd.DataFrame(normalizationScaling, columns=user1_score.drop(['item_id'], axis=1).columns)

In [80]:
normalizationScaling.describe()

,score
count,27204.000000
mean,0.682650
std,0.106023
min,0.000000
25%,0.611653
50%,0.675200
75%,0.757618
max,1.000000


In [84]:
print(type(normalizationScaling))

<class 'pandas.core.frame.DataFrame'>


In [85]:
normalizationScaling.head()

,score
0,0.969373
1,0.568861
2,0.635729
3,0.821039
4,0.589317


In [88]:
normalizationScaling = normalizationScaling.mul(10)

In [89]:
normalizationScaling.head()

,score
0,9.693735
1,5.688610
2,6.357288
3,8.210392
4,5.893168


In [ ]:
# model.predict(user_ids=np.array([0,0,0,0,0]), item_ids=np.array([0,1,2,5,100]), 
#                     item_features=item_features, user_features=user_features)

In [98]:
test = pd.read_csv('../../data/' + 'test_ratings.csv')
test

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [108]:
test.loc[(test['user_id']==11676)].shape

(1419, 3)

In [93]:
# userid_unique = df['user_id'].unique()
# book_unique = df['isbn'].unique()
# print(len(userid_unique))

42668


In [94]:
# user_to_idx = {v:k for k,v in enumerate(userid_unique)}
# book_to_idx = {v:k for k,v in enumerate(book_unique)}

In [95]:
# test['uid'] = test['user_id'].map(user_to_idx)
# test['iid'] = test['isbn'].map(book_to_idx)
# test.head()

,user_id,isbn,rating,uid,iid
0,11676,0002005018,0,NaN,0.0
1,116866,0002005018,0,9649.0,0.0
2,152827,0060973129,0,19998.0,1.0
3,157969,0374157065,0,11968.0,2.0
4,67958,0399135782,0,24769.0,3.0


In [113]:
test['user_id'].tolist()

[11676,
 116866,
 152827,
 157969,
 67958,
 213191,
 245827,
 11676,
 263256,
 226745,
 7352,
 12923,
 13552,
 21264,
 31299,
 64535,
 70666,
 75819,
 79740,
 85534,
 100393,
 108143,
 116127,
 122251,
 128463,
 131703,
 132144,
 134863,
 136707,
 143939,
 144062,
 150400,
 154730,
 165884,
 187533,
 187944,
 217516,
 227392,
 240567,
 245616,
 250101,
 258884,
 259223,
 273979,
 6242,
 8570,
 11104,
 11676,
 36197,
 36907,
 80837,
 102359,
 144803,
 165319,
 174633,
 221232,
 252627,
 271135,
 271144,
 11676,
 79089,
 271622,
 141518,
 157118,
 14,
 4784,
 8453,
 11554,
 12264,
 13088,
 13221,
 14049,
 14461,
 20147,
 21404,
 24664,
 28091,
 28735,
 32329,
 33535,
 37538,
 40841,
 41596,
 45666,
 46398,
 49954,
 51803,
 55894,
 59994,
 60633,
 61028,
 64819,
 65145,
 65370,
 67636,
 67704,
 69378,
 70153,
 71162,
 71464,
 72030,
 74910,
 79738,
 83469,
 84682,
 89867,
 91254,
 92727,
 98655,
 104268,
 111433,
 114434,
 121070,
 127619,
 128447,
 130915,
 133645,
 136139,
 136237,
 141

In [ ]:
# ratings_source = [(data['uid'][i], data['iid'][i]) for i in range(data.shape[0])]

In [112]:
scores = []

for user_id,isbn in zip(test['user_id'].tolist(), test['isbn'].tolist()):
    user_ids = np.array([user_id])
    isbns = np.array([isbn])
    s = model.predict(user_ids, isbns, 
                      item_features=item_features, user_features=user_features)
    scores.append([isbns[0], s[0]])

Exception: Number of item feature rows does not equal the number of items

In [ ]:
# print(f'--------------- SAVE ightfm_model PREDICT ---------------')
# submission = pd.read_csv('../../data/' + 'sample_submission.csv')
# submission['rating'] = predicts

# now = time.localtime()
# now_date = time.strftime('%Y%m%d', now)
# now_hour = time.strftime('%X', now)
# save_time = now_date + '_' + now_hour.replace(':', '')
# submission.to_csv('submit/{}_{}.csv'.format(save_time, 'lightfm_model'), index=False)


In [ ]:
# from lightfm.evaluation import auc_score, precision_at_k, recall_at_k

In [96]:
# auc_score(model, train, 
#         item_features=item_features, 
#         user_features=user_features).mean()

NameError: name 'auc_score' is not defined

In [ ]:
# test_auc = auc_score(model, test, 
#         item_features=item_features, 
#         user_features=user_features).mean()

In [ ]:
# test_precision = precision_at_k(model, test, k=100, 
#                                 item_features=item_features,
#                                 user_features=user_features).mean()

In [ ]:
# test_recall = recall_at_k(model, test, k=100, 
#                           item_features=item_features,
#                           user_features=user_features).mean()

In [ ]:
# test_auc, test_precision, test_recall